In [11]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import cartopy.io.shapereader as shpreader
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import matplotlib.patches as mpatches
import geopandas as gpd

import psycopg2
from sqlalchemy import create_engine

https://www.census.gov/topics/public-sector/voting/data/tables.2010.List_1863097513.html#list-tab-List_1863097513

In [12]:
# read in the Excel file and specify the sheet name
df_votes_2018 = pd.read_excel('Election_Results/vote2018.xlsx', sheet_name='Table 4a', header=[4])

# display the resulting dataframe
df_votes_2018.head()

Table 4a


STATE  Total population  Total citizen population   
0            NaN               NaN                       NaN  \
1  UNITED STATES          249748.0                  228832.0   
2        ALABAMA            3753.0                    3609.0   
3         ALASKA             523.0                     497.0   
4        ARIZONA            5361.0                    4757.0   

         Registered                   Unnamed: 4         Unnamed: 5   
0  Total registered  Percent registered\n(Total)  Margin of error 1  \
1            153066                         61.3                0.3   
2              2490                         66.4                2.5   
3               337                         64.4                2.7   
4              3262                         60.8                2.2   

                      Unnamed: 6         Unnamed: 7        Voted   
0  Percent registered\n(Citizen)  Margin of error 1  Total voted  \
1                           66.9                0.3       122281   
2                             69                2.5         1830   
3                           67.7                2.7          263   
4                           68.6                2.2         2800   

               Unnamed: 9        Unnamed: 10               Unnamed: 11   
0  Percent voted\n(Total)  Margin of error 1  Percent voted\n(Citizen)  \
1                      49                0.3                      53.4   
2                    48.8                2.7                      50.7   
3                    50.2                2.9                      52.8   
4                    52.2                2.3                      58.9   

         Unnamed: 12  
0  Margin of error 1  
1                0.3  
2                2.7  
3                2.9  
4                2.4

In [13]:
# select the columns wanted and rename them
df_selection = df_votes_2018[['STATE', 'Total population', 'Registered', 'Voted']]
df_votes = df_selection.rename(columns={
    'STATE': 'State',
    'Total population': 'Population',
    'Registered': 'Registered'
})
# add year and office columns
df_votes['Year'] = 2018

# Arrange columns: Move the Year and Office columns to the front
new_cols = ['Year', 'State', 'Population', 'Registered', 'Voted']

df_votes = df_votes.iloc[1:]  # remove the first row
df_votes = df_votes.iloc[:-7] # removes last 7 rows of text

# Save the first row as a separate DataFrame
first_row = df_votes.iloc[[0]].copy()

# Remove the first row from the original DataFrame
df_votes = df_votes.iloc[1:].copy()

# Append the first row to the end of the DataFrame
df_votes = pd.concat([df_votes, first_row])

df_votes = df_votes.reset_index(drop=True)  # reset the index
df_votes = df_votes.reindex(columns=new_cols) # set year column in front

df_votes
# values are in the thousands

Year                 State  Population Registered   Voted
0   2018               ALABAMA      3753.0       2490    1830
1   2018                ALASKA       523.0        337     263
2   2018               ARIZONA      5361.0       3262    2800
3   2018              ARKANSAS      2261.0       1262     919
4   2018            CALIFORNIA     30243.0      15690   13240
5   2018              COLORADO      4353.0       2645    2342
6   2018           CONNECTICUT      2834.0       1726    1370
7   2018              DELAWARE       756.0        472     369
8   2018  DISTRICT OF COLUMBIA       567.0        397     313
9   2018               FLORIDA     16845.0       9435    7918
10  2018               GEORGIA      7850.0       4840    4084
11  2018                HAWAII      1057.0        523     427
12  2018                 IDAHO      1299.0        743     587
13  2018              ILLINOIS      9732.0       6068    4740
14  2018               INDIANA      5006.0       3131    2364
15  2018                  IOWA      2376.0       1658    1335
16  2018                KANSAS      2149.0       1449    1152
17  2018              KENTUCKY      3370.0       2389    1746
18  2018             LOUISIANA      3458.0       2263    1656
19  2018                 MAINE      1074.0        828     693
20  2018              MARYLAND      4666.0       3095    2320
21  2018         MASSACHUSETTS      5460.0       3345    2731
22  2018              MICHIGAN      7657.0       5453    4418
23  2018             MINNESOTA      4238.0       3000    2523
24  2018           MISSISSIPPI      2194.0       1599    1180
25  2018              MISSOURI      4676.0       3299    2509
26  2018               MONTANA       822.0        579     518
27  2018              NEBRASKA      1428.0        883     676
28  2018                NEVADA      2324.0       1277    1006
29  2018         NEW HAMPSHIRE      1080.0        726     576
30  2018            NEW JERSEY      7009.0       4297    3384
31  2018            NEW MEXICO      1576.0        916     715
32  2018              NEW YORK     15478.0       8553    6775
33  2018        NORTH CAROLINA      7911.0       5160    3899
34  2018          NORTH DAKOTA       560.0        397     335
35  2018                  OHIO      8873.0       6062    4538
36  2018              OKLAHOMA      2868.0       1777    1350
37  2018                OREGON      3293.0       2274    1918
38  2018          PENNSYLVANIA      9928.0       6469    5173
39  2018          RHODE ISLAND       828.0        532     403
40  2018        SOUTH CAROLINA      3914.0       2430    1836
41  2018          SOUTH DAKOTA       648.0        429     331
42  2018             TENNESSEE      5202.0       3183    2487
43  2018                 TEXAS     21064.0      11634    8886
44  2018                  UTAH      2247.0       1443    1214
45  2018               VERMONT       503.0        343     273
46  2018              VIRGINIA      6386.0       4159    3319
47  2018            WASHINGTON      5775.0       3852    3234
48  2018         WEST VIRGINIA      1406.0        892     610
49  2018             WISCONSIN      4436.0       3129    2776
50  2018               WYOMING       430.0        268     220
51  2018         UNITED STATES    249748.0     153066  122281

In [14]:
state_abbr = {
    'ALABAMA': 'AL',
    'ALASKA': 'AK',
    'ARIZONA': 'AZ',
    'ARKANSAS': 'AR',
    'CALIFORNIA': 'CA',
    'COLORADO': 'CO',
    'CONNECTICUT': 'CT',
    'DELAWARE': 'DE',
    'DISTRICT OF COLUMBIA': 'DC',
    'FLORIDA': 'FL',
    'GEORGIA': 'GA',
    'HAWAII': 'HI',
    'IDAHO': 'ID',
    'ILLINOIS': 'IL',
    'INDIANA': 'IN',
    'IOWA': 'IA',
    'KANSAS': 'KS',
    'KENTUCKY': 'KY',
    'LOUISIANA': 'LA',
    'MAINE': 'ME',
    'MARYLAND': 'MD',
    'MASSACHUSETTS': 'MA',
    'MICHIGAN': 'MI',
    'MINNESOTA': 'MN',
    'MISSISSIPPI': 'MS',
    'MISSOURI': 'MO',
    'MONTANA': 'MT',
    'NEBRASKA': 'NE',
    'NEVADA': 'NV',
    'NEW HAMPSHIRE': 'NH',
    'NEW JERSEY': 'NJ',
    'NEW MEXICO': 'NM',
    'NEW YORK': 'NY',
    'NORTH CAROLINA': 'NC',
    'NORTH DAKOTA': 'ND',
    'OHIO': 'OH',
    'OKLAHOMA': 'OK',
    'OREGON': 'OR',
    'PENNSYLVANIA': 'PA',
    'RHODE ISLAND': 'RI',
    'SOUTH CAROLINA': 'SC',
    'SOUTH DAKOTA': 'SD',
    'TENNESSEE': 'TN',
    'TEXAS': 'TX',
    'UTAH': 'UT',
    'VERMONT': 'VT',
    'VIRGINIA': 'VA',
    'WASHINGTON': 'WA',
    'WEST VIRGINIA': 'WV',
    'WISCONSIN': 'WI',
    'WYOMING': 'WY',
    'UNITED STATES': 'US'
}



In [15]:
df_votes['State'] = df_votes['State'].map(state_abbr)
df_votes

Year State  Population Registered   Voted
0   2018    AL      3753.0       2490    1830
1   2018    AK       523.0        337     263
2   2018    AZ      5361.0       3262    2800
3   2018    AR      2261.0       1262     919
4   2018    CA     30243.0      15690   13240
5   2018    CO      4353.0       2645    2342
6   2018    CT      2834.0       1726    1370
7   2018    DE       756.0        472     369
8   2018    DC       567.0        397     313
9   2018    FL     16845.0       9435    7918
10  2018    GA      7850.0       4840    4084
11  2018    HI      1057.0        523     427
12  2018    ID      1299.0        743     587
13  2018    IL      9732.0       6068    4740
14  2018    IN      5006.0       3131    2364
15  2018    IA      2376.0       1658    1335
16  2018    KS      2149.0       1449    1152
17  2018    KY      3370.0       2389    1746
18  2018    LA      3458.0       2263    1656
19  2018    ME      1074.0        828     693
20  2018    MD      4666.0       3095    2320
21  2018    MA      5460.0       3345    2731
22  2018    MI      7657.0       5453    4418
23  2018    MN      4238.0       3000    2523
24  2018    MS      2194.0       1599    1180
25  2018    MO      4676.0       3299    2509
26  2018    MT       822.0        579     518
27  2018    NE      1428.0        883     676
28  2018    NV      2324.0       1277    1006
29  2018    NH      1080.0        726     576
30  2018    NJ      7009.0       4297    3384
31  2018    NM      1576.0        916     715
32  2018    NY     15478.0       8553    6775
33  2018    NC      7911.0       5160    3899
34  2018    ND       560.0        397     335
35  2018    OH      8873.0       6062    4538
36  2018    OK      2868.0       1777    1350
37  2018    OR      3293.0       2274    1918
38  2018    PA      9928.0       6469    5173
39  2018    RI       828.0        532     403
40  2018    SC      3914.0       2430    1836
41  2018    SD       648.0        429     331
42  2018    TN      5202.0       3183    2487
43  2018    TX     21064.0      11634    8886
44  2018    UT      2247.0       1443    1214
45  2018    VT       503.0        343     273
46  2018    VA      6386.0       4159    3319
47  2018    WA      5775.0       3852    3234
48  2018    WV      1406.0        892     610
49  2018    WI      4436.0       3129    2776
50  2018    WY       430.0        268     220
51  2018    US    249748.0     153066  122281

In [16]:
# Calculate the normalized voter count
df_votes['Norm_Voter_Reg'] = df_votes['Voted'] / df_votes['Registered']
df_votes['Norm_Voter_Pop'] = df_votes['Voted'] / df_votes['Population']
df_votes

Year State  Population Registered   Voted Norm_Voter_Reg Norm_Voter_Pop
0   2018    AL      3753.0       2490    1830        0.73494        0.48761
1   2018    AK       523.0        337     263       0.780415       0.502868
2   2018    AZ      5361.0       3262    2800       0.858369       0.522291
3   2018    AR      2261.0       1262     919       0.728209       0.406457
4   2018    CA     30243.0      15690   13240        0.84385       0.437787
5   2018    CO      4353.0       2645    2342       0.885444        0.53802
6   2018    CT      2834.0       1726    1370       0.793743       0.483416
7   2018    DE       756.0        472     369        0.78178       0.488095
8   2018    DC       567.0        397     313       0.788413       0.552028
9   2018    FL     16845.0       9435    7918       0.839216        0.47005
10  2018    GA      7850.0       4840    4084       0.843802       0.520255
11  2018    HI      1057.0        523     427       0.816444       0.403974
12  2018    ID      1299.0        743     587        0.79004       0.451886
13  2018    IL      9732.0       6068    4740       0.781147       0.487053
14  2018    IN      5006.0       3131    2364        0.75503       0.472233
15  2018    IA      2376.0       1658    1335       0.805187       0.561869
16  2018    KS      2149.0       1449    1152       0.795031       0.536063
17  2018    KY      3370.0       2389    1746        0.73085       0.518101
18  2018    LA      3458.0       2263    1656       0.731772        0.47889
19  2018    ME      1074.0        828     693       0.836957       0.645251
20  2018    MD      4666.0       3095    2320       0.749596       0.497214
21  2018    MA      5460.0       3345    2731       0.816442       0.500183
22  2018    MI      7657.0       5453    4418       0.810196       0.576988
23  2018    MN      4238.0       3000    2523          0.841       0.595328
24  2018    MS      2194.0       1599    1180       0.737961        0.53783
25  2018    MO      4676.0       3299    2509       0.760533        0.53657
26  2018    MT       822.0        579     518       0.894646        0.63017
27  2018    NE      1428.0        883     676       0.765572       0.473389
28  2018    NV      2324.0       1277    1006       0.787784       0.432874
29  2018    NH      1080.0        726     576       0.793388       0.533333
30  2018    NJ      7009.0       4297    3384       0.787526       0.482808
31  2018    NM      1576.0        916     715       0.780568        0.45368
32  2018    NY     15478.0       8553    6775        0.79212       0.437718
33  2018    NC      7911.0       5160    3899        0.75562       0.492858
34  2018    ND       560.0        397     335       0.843829       0.598214
35  2018    OH      8873.0       6062    4538       0.748598       0.511439
36  2018    OK      2868.0       1777    1350       0.759707       0.470711
37  2018    OR      3293.0       2274    1918       0.843448       0.582448
38  2018    PA      9928.0       6469    5173        0.79966       0.521052
39  2018    RI       828.0        532     403       0.757519       0.486715
40  2018    SC      3914.0       2430    1836       0.755556       0.469085
41  2018    SD       648.0        429     331       0.771562       0.510802
42  2018    TN      5202.0       3183    2487       0.781338       0.478085
43  2018    TX     21064.0      11634    8886       0.763796       0.421857
44  2018    UT      2247.0       1443    1214       0.841303       0.540276
45  2018    VT       503.0        343     273       0.795918       0.542744
46  2018    VA      6386.0       4159    3319       0.798028       0.519731
47  2018    WA      5775.0       3852    3234       0.839564           0.56
48  2018    WV      1406.0        892     610       0.683857       0.433855
49  2018    WI      4436.0       3129    2776       0.887184       0.625789
50  2018    WY       430.0        268     220       0.820896       0.511628
51  2018    US    249748.0     153066  122281       

# Add to sql db

In [17]:
# import psycopg2
# from sqlalchemy import create_engine

# Replace the values below with your database credentials
DATABASE = "electiondb"
USER = "postgres"
PASSWORD = "YourPassword"
HOST = "localhost"
PORT = "5432"

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    database=DATABASE,
    user=USER,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

# Create a sqlalchemy engine
engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

# Upload the data frames to the database using the voters table
df_votes.to_sql("voters", engine, if_exists="append", index=False)
print("Votes table loaded successfully")

# Commit the changes to the database
conn.commit()

# Close the cursor and database connection
cursor.close()
conn.close()


Votes table loaded successfully


In [18]:
# import psycopg2
# import pandas as pd

# Replace the values below with your database credentials
DATABASE = "electiondb"
USER = "postgres"
PASSWORD = "YourPassword"
HOST = "localhost"
PORT = "5432"

# Connect to the PostgreSQL database
conn = psycopg2.connect(database=DATABASE, user=USER, password=PASSWORD, host=HOST, port=PORT)

# Query the table and store the results in a Pandas dataframe
df = pd.read_sql_query("SELECT * FROM voters", conn)

# Print the dataframe to verify that it contains data
print(df)

# Close the database connection
conn.close()
# gives 1128 rows for 2020

     Year State  Population  Registered   Voted  Norm_Voter_Reg   
0    2020    AL      3769.0        2527    2247        0.889197  \
1    2020    AK       528.0         383     330        0.861619   
2    2020    AZ      5638.0        3878    3649        0.940949   
3    2020    AR      2283.0        1361    1186        0.871418   
4    2020    CA     30342.0       18001   16893        0.938448   
..    ...   ...         ...         ...     ...             ...   
99   2018    WA      5775.0        3852    3234        0.839564   
100  2018    WV      1406.0         892     610        0.683857   
101  2018    WI      4436.0        3129    2776        0.887184   
102  2018    WY       430.0         268     220        0.820896   
103  2018    US    249748.0      153066  122281        0.798878   

     Norm_Voter_Pop  
0          0.596179  
1          0.625000  
2          0.647215  
3          0.519492  
4          0.556753  
..              ...  
99         0.560000  
100        0.433855